Generating features for images
-----------------------------
Load the VGGNet model with the pre-trained weights. Get the files necessary for VGGNet from [here](https://github.com/szagoruyko/loadcaffe). We use the model called VGG ILSVRC-2014 19-layer. 

In [1]:
require 'cunn'
require 'loadcaffe'
require 'optim'
vgg = loadcaffe.load('VGGNet/VGG_ILSVRC_19_layers_deploy.prototxt.txt','VGGNet/VGG_ILSVRC_19_layers.caffemodel')

Successfully loaded VGGNet/VGG_ILSVRC_19_layers.caffemodel


conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3


conv2_2: 128 128 3 3
conv3_1: 256 128 3 3
conv3_2: 256 256 3 3


conv3_3: 256 256 3 3


conv3_4: 256 256 3 3


...

conv5_3: 512 512 3 3


conv5_4: 512 512 3 3


fc6: 1 1 25088 4096


fc7: 1 1 4096 4096


fc8: 1 1 4096 1000


Transfer this model to cuda

In [2]:
vgg:cuda();

Load all the meme categories.

In [3]:
require 'lfs'
classes = {}
for file in lfs.dir("Meme_generator/Data") do
    if lfs.attributes(file, "mode") ~= "directory" then 
        table.insert(classes, file)
    end
end

In [4]:
classes

{
  1 : Brian-Williams-Was-There
  2 : Yall-Got-Any-More-Of
  3 : The-Critic
  4 : Serious-Xzibit
  5 : Not-Okay-Rage-Face
  6 : Lewandowski-E-Reus
  7 : Frowning-Nun
  8 : Original-Bad-Luck-Brian
  9 : But-Thats-None-Of-My-Business
  10 : Scared-Cat
  11 : Fim-De-Semana
  12 : Barack-And-Kumar-2013
  13 : Derpina
  14 : Simba-Shadowy-Place
  15 : The-Bobs
  16 : Overly-Attached-Girlfriend
  17 : Successful-Black-Man
  18 : Hedonism-Bot
  19 : Smirk-Rage-Face
  20 : Officer-Cartman
  21 : Luiz-Fabiano
  22 : Bothered-Bond
  23 : Slenderman
  24 : Advice-Yoda
  25 : Hey-Internet
  26 : So-Much-Drama
  27 : House-Bunny
  28 : RPG-Fan
  29 : Too-Damn-High
  30 : Metal-Jesus
  31 : Dexter
  32 : Sweaty-Concentrated-Rage-Face
  33 : Rebecca-Black
  34 : Dr-Evil
  35 : Tough-Guy-Wanna-Be
  36 : Bill-OReilly
  37 : Surprised-Koala
  38 : Ermahgerd-Berks
  39 : Neil-deGrasse-Tyson
  40 : Derp
  41 : You-Should-Feel-Bad-Zoidberg
  42 : Darti-Boy
  43 : Malicious-Advice-Mallard
  44 : Saw-Fulla


  120 : Oku-Manami
  121 : If-You-Know-What-I-Mean-Bean
  122 : Bah-Humbug
  123 : Lion-King
  124 : You-Dont-Want-No-Part-Of-This
  125 : Third-World-Success-Kid
  126 : Laughing-Goat
  127 : Britney-Spears
  128 : Bad-Luck-Brian
  129 : Bebo
  130 : DoucheBag-DJ
  131 : Monkey-Business
  132 : Ghetto-Jesus
  133 : Imagination-Spongebob
  134 : Really-Evil-College-Teacher
  135 : Angry-Baby
  136 : Chubby-Bubbles-Girl
  137 : One-Does-Not-Simply
  138 : Matrix-Morpheus
  139 : Right-In-The-Childhood
  140 : Marvel-Civil-War-2
  141 : Inception
  142 : Onde
  143 : Crazy-Girlfriend-Praying-Mantis
  144 : Error-404
  145 : Laughing-Men-In-Suits
  146 : Sexy-Railroad-Spiderman
  147 : Forever-Alone
  148 : Look-At-All-These
  149 : Surprised-Coala
  150 : Surprized-Vegeta
  151 : Say-That-Again-I-Dare-You
  152 : Innocent-Sasha
  153 : Sarcastic-Anthony
  154 : WTF
  155 : Original-I-Lied
  156 : Reimu-Hakurei
  157 : He-Will-Never-Get-A-Girlfriend
  158 : Harry-Potter-Ok
  159 : City-Be

  281 : Peter-Parker-Cry
  282 : Oprah-You-Get-A
  283 : Facepalm-Bear
  284 : Nice-Guy-Loki
  285 : High-Dog
  286 : Comic-Book-Guy
  287 : Ermahgerd-Beyonce
  288 : Cute-Cat
  289 : Papa-Fking-John
  290 : Philosoraptor
  291 : Futurama-Leela
  292 : Paul-Wonder-Years
  293 : Headbanzer
  294 : Cool-Cat-Stroll
  295 : No-Bullshit-Business-Baby
  296 : Ancient-Aliens
  297 : Rick-and-Carl
  298 : Oprah-You-Get-A-Car-Everybody-Gets-A-Car
  299 : Bart-Simpson-Peeking
  300 : Memeo
  301 : So-Many-Shirts
  302 : Relaxed-Office-Guy
  303 : Dr-Crane
  304 : PTSD-Clarinet-Boy
  305 : Anti-Joke-Chicken
  306 : Spiderman-Peter-Parker
  307 : Chainsaw-Bear
  308 : Richard-Benson
  309 : Grumpy-Cat-Halloween
  310 : Sidious-Error
  311 : Okay-Guy-Rage-Face2
  312 : FFFFFFFUUUUUUUUUUUU
  313 : Sigmund-Freud
  314 : Dancing-Trollmom
  315 : Secure-Parking
  316 : Why-Is-The-Rum-Gone
  317 : Slowpoke
  318 : Bear-Grylls
  319 : Yao-Ming
  320 : Bad-Pun-Anna-Kendrick
  321 : Interupting-Kanye
  322

-Kid-Girl
  446 : Fast-Furious-Johnny-Tran
  447 : I-See-Dead-People
  448 : I-Lied-2
  449 : Confused-Cam
  450 : Oh-No
  451 : Helpful-Tyler-Durden
  452 : Lethal-Weapon-Danny-Glover
  453 : Bad-Luck-Hannah
  454 : Unpopular-Opinion-Puffin
  455 : And-everybody-loses-their-minds
  456 : First-World-Problems-Cat
  457 : Angry-Bride
  458 : College-Freshman
  459 : Larry-The-Cable-Guy
  460 : Bill-Murray-Golf
  461 : How-About-No-Bear
  462 : Rena-Matsui
  463 : Afraid-To-Ask-Andy
  464 : High-Expectations-Asian-Father
  465 : Be-Like-Bill
  466 : DJ-Pauly-D
  467 : Captain-Picard-Facepalm
  468 : The-Rock-It-Doesnt-Matter
  469 : Barbosa-And-Sparrow
  470 : Evil-Otter
  471 : Fini
  472 : Kim-Jong-Un-Sad
  473 : Mr-Mackey
  474 : I-Will-Find-You-And-Kill-You
  475 : Penguin-Gang
  476 : Close-Enough
  477 : Good-Guy-Greg
  478 : Fat-Cat
  479 : Shutup-Batty-Boy
  480 : Big-Bird-And-Snuffy
  481 : Casper
  482 : Big-Ego-Man
  483 : Kim-Jong-Il-Y-U-No
  484 : Gandhi
  485 : Alien-Meetin

 Peter-Griffin-News
  611 : Scrooge-McDuck
  612 : Storytelling-Grandpa
  613 : Burn-Kitty
  614 : Scumbag-Steve
  615 : Do-I-Care-Doe
  616 : Baby-Insanity-Wolf
  617 : Hardworking-Guy
  618 : Grumpy-Cats-Father
  619 : Guinness-World-Record
  620 : Good-Guy-Putin
  621 : Zombie-Overly-Attached-Girlfriend
  622 : Stop-Cop
  623 : Bad-Luck-Bear
  624 : Dwight-Schrute-2
  625 : Blue-Futurama-Fry
  626 : Bill-Nye-The-Science-Guy
  627 : TED
  628 : Men-Laughing
  629 : John-Riley-Condescension
  630 : Jon-Stewart-Skeptical
  631 : Life-Sucks
  632 : First-Day-On-The-Internet-Kid
  633 : Bad-Wife-Worse-Mom
  634 : Solemn-Lumberjack
  635 : Multi-Doge
  636 : Hipster-Barista
  637 : Excited-Cat
  638 : Small-Face-Romney
  639 : Oblivious-Hot-Girl
  640 : Aw-Yeah-Rage-Face
  641 : Billy-Graham-Mitt-Romney
  642 : Socially-Awkward-Awesome-Penguin
  643 : Blank-Blue-Background
  644 : Father-Ted
  645 : Since-When-Were-You-Under-The-Impression
  646 : Rich-Raven
  647 : Steve-Harvey
  648 : Y

achelor-Frog
  769 : Mad-Money-Jim-Cramer
  770 : 1950s-Middle-Finger
  771 : Unwanted-House-Guest
  772 : Cool-Story-Bro
  773 : Police-Officer-Testifying
  774 : Hawkward
  775 : Sudden-Disgust-Danny
  776 : 10-Guy
  777 : Obama-No-Listen
  778 : Frustrated-Boromir
  779 : Nabilah-Jkt48
  780 : Troll-Face
}


Function to pass an image through the VGGNet. 

In [11]:
function forward_image(vgg, image_path)
    im = image.load(image_path)
    siz = im:size()
    if siz[1] > 3 then
        im = im[{{1,3}}]
    elseif siz[1] < 3 then
        newim = torch.Tensor(3, siz[2], siz[3])
        newim[1] = im[1]
        newim[2] = im[1]
        newim[3] = im[1]
        im = newim
    end
    scaled = image.scale(im,224,224)
    scaled = scaled:cuda()
    scaled = scaled*256
    scaled[1] = scaled[1] - 123.68
    scaled[2] = scaled[2] - 116.779
    scaled[3] = scaled[3] - 103.939
    return vgg:forward(scaled)
end

Limit the number of images per category for faster training.

In [7]:
img_per_class = 50
print(#classes)

780	


Create empty tensors for containing the input features and labels.

In [8]:
input = torch.Tensor(img_per_class*#classes, 4096)
labels = torch.LongTensor(img_per_class*#classes)
print(img_per_class*#classes)

39000	


Shift input to GPU.

In [9]:
input = input:cuda()

Pass each image from the dataset through the VGGNet. Get the output at the last but one layer. 

In [12]:
for index, class in pairs(classes) do
    for i=0,img_per_class-1 do
        actual_index = (index-1)*img_per_class+i+1
        if actual_index%1000 == 0 then
            print(actual_index)
        end
        forward_image(vgg, "Meme_generator/Data/"..class.."/"..i..".jpg")
        input[{actual_index}] = vgg.modules[43].output
        labels[{actual_index}] = index
    end
    collectgarbage()
end


1000	


2000	


3000	


4000	


5000	


...

35000	


36000	


37000	


38000	


39000	


Save these features to a file to be used for training on meme categories.

In [17]:
input = input:float()
data = {input, labels, classes}
torch.save("meme_features.t7", data)